In [57]:
import torch
model, alphabet = torch.hub.load("facebookresearch/esm", "esm1_t34_670M_UR50S")
import esm

Using cache found in /home/cyu7/.cache/torch/hub/facebookresearch_esm_master


In [15]:
import numpy as np
import pandas as pd

In [16]:
pfamA_balanced = pd.read_csv("../../data/pfamA_motors_balanced.csv")
pfamA_balanced.head()

,Unnamed: 0,id,description,seq,pfamA_acc,clan
0,167044,U5C9Z2_9BACT/7-195,U5C9Z2_9BACT/7-195 U5C9Z2.1 PF00814.26;TsaD;,HQDNVHARSLMGLVRNVFEQAGLEKTALDAVAVSSGPGSYTGLRIG...,PF00814,actin_like
1,125588,A0A2C9ZZA3_9GAMM/2-187,A0A2C9ZZA3_9GAMM/2-187 A0A2C9ZZA3.1 PF03309.15...,ILLIDAGNTAIKLAVSDEHNQPTLIKQVQLCWSKISLVLYSCVKHS...,PF03309,actin_like
2,161982,W6LR88_9GAMM/14-234,W6LR88_9GAMM/14-234 W6LR88.1 PF05134.14;T2SSL;,DQAEWLLADGTVQHGTLTELVPHVIGARLLLIAPGERLTLHRVPLP...,PF05134,actin_like
3,22085,A0A2H3ZDD2_PHODC/25-338,A0A2H3ZDD2_PHODC/25-338 A0A2H3ZDD2.1 PF01869.2...,LGLDGGTTSTVCVCLPAAMPLASALPDPLPILSRAVAGCSNHNSVG...,PF01869,actin_like
4,10493,Q75BT7_ASHGO/309-399,Q75BT7_ASHGO/309-399 Q75BT7.1 PF00022.20;Actin;,AGLPSATPEQIHAALLTNVVIIGGTSLLQGLEQRLVNDLSLQFPQY...,PF00022,actin_like


In [17]:
pfamA_random = pd.read_csv("../../data/pfamA_random_201027.csv")
pfamA_random.head()

,Unnamed: 0,description,seq,pfamA_acc,pfamA_name
0,2966648,A0A1Y3GBD4_9EURY/5-68 A0A1Y3GBD4.1 PF01918.22;...,NVVYVGNKEVMSYVLAVTTQFNEGSDEVVIKARGRAISTAVDTAEV...,PF01918,Alba
1,2966649,G0V5M1_NAUCC/40-110 G0V5M1.1 PF01918.22;Alba;,KRVCITKNANIKHTVDKLQGLLESTEGFVCLEAHGPHLQKMLSILE...,PF01918,Alba
2,2966650,A0A2G2W1S6_CAPBA/19-84 A0A2G2W1S6.1 PF01918.22...,NEIRVTAQGLIRNYISYATTLLQDQRTNEVVLKAMGQAISKTVAIA...,PF01918,Alba
3,2966651,G2QUI4_THITE/71-147 G2QUI4.1 PF01918.22;Alba;,KPMSVMPSTSIGKHVDRALEHLGRFSAWDTSVLPGVVLLCAKSSAS...,PF01918,Alba
4,2966652,A0A0E0K979_ORYPU/20-85 A0A0E0K979.1 PF01918.22...,NEIRITTQGLIRNYVTYATSLLQEKRVKEIVLKAMGQAISKTVAIA...,PF01918,Alba


In [18]:
batch_converter = alphabet.get_batch_converter()

In [19]:
model.cuda()

ProteinBertModel(
  (embed_tokens): Embedding(35, 1280, padding_idx=1)
  (embed_positions): PositionalEmbedding()
  (layers): ModuleList(
    (0): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
      )
      (self_attn_layer_norm): BertLayerNorm()
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): BertLayerNorm()
    )
    (1): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_featu

In [21]:
# Prepare data (two protein sequences)
# data = [("protein1", "HQDNVHARSLMGLVRNVFEQAGLEKTALDAVAVSSGPG"), ("protein2", "DAGNTAIKLAVSDEHNQPTLIKQVQLCWSKISLVLYSCVKHS")]


# print(batch_labels)
# print(batch_strs)
# print(batch_tokens)
# Extract per-residue embeddings (on CPU)
def esm_embed(df):
    sequence_embeddings = []
    for i in range(pfamA_balanced.shape[0]):
        if (i%10)==0:
            print(i)
        data = [(pfamA_balanced.iloc[i,1], pfamA_balanced.iloc[i,3])]
        batch_labels, batch_strs, batch_tokens = batch_converter(data)
        print(batch_labels)
        print(batch_strs)
        print(batch_tokens)
        
        
        with torch.no_grad():
            results = model(batch_tokens.to('cuda'), repr_layers=[34])
            # last layer
            token_embeddings = results["representations"][34]
            seq = pfamA_balanced.iloc[i,3]
            sequence_embeddings.append(token_embeddings[0, 1:len(seq) + 1].mean(0).cpu().detach().numpy())
        break
#     print(np.array(sequence_embeddings).shape)
#     np.save("../../data/balanced_esm.npy", np.array(sequence_embeddings))
    return 



In [22]:
esm_embed(pfamA_balanced)

0
['U5C9Z2_9BACT/7-195']
['HQDNVHARSLMGLVRNVFEQAGLEKTALDAVAVSSGPGSYTGLRIGVSVAKGLAYALDKPVIGVGTLEALAFRAIPFSDSTDTIIPMLDARRMEVYALVMDGLGDTLISPQPFILEDNPFMEYLEKGKVFFLGDGVPKSKEILSHPNSRFVPLFNSSQSIGELAYKKFLKADFESLAYFEPNYIKEFRI']
tensor([[32, 21, 16, 13, 17,  7, 21,  5, 10,  8,  4, 20,  6,  4,  7, 10, 17,  7,
         18,  9, 16,  5,  6,  4,  9, 15, 11,  5,  4, 13,  5,  7,  5,  7,  8,  8,
          6, 14,  6,  8, 19, 11,  6,  4, 10, 12,  6,  7,  8,  7,  5, 15,  6,  4,
          5, 19,  5,  4, 13, 15, 14,  7, 12,  6,  7,  6, 11,  4,  9,  5,  4,  5,
         18, 10,  5, 12, 14, 18,  8, 13,  8, 11, 13, 11, 12, 12, 14, 20,  4, 13,
          5, 10, 10, 20,  9,  7, 19,  5,  4,  7, 20, 13,  6,  4,  6, 13, 11,  4,
         12,  8, 14, 16, 14, 18, 12,  4,  9, 13, 17, 14, 18, 20,  9, 19,  4,  9,
         15,  6, 15,  7, 18, 18,  4,  6, 13,  6,  7, 14, 15,  8, 15,  9, 12,  4,
          8, 21, 14, 17,  8, 10, 18,  7, 14,  4, 18, 17,  8,  8, 16,  8, 12,  6,
          9,  4,  5, 19, 15, 15, 18,  4, 15,  5, 13,

In [48]:
alphabet.mask_idx

33

In [28]:
batch_tokens.size(1)

207

In [37]:
batch_tokens[:,3] = 34
seq_len = batch_tokens.size(1)

In [38]:
shuffled_seqind = torch.randperm(seq_len)+1

In [51]:
shuffled_seqind[0:5] = torch.tensor([1,2,3,4,5])
shuffled_seqind

tensor([  1,   2,   3,   4,   5,  44,  65, 112, 171,  56, 168,  34, 164, 131,
        180,  54, 126, 181, 127, 187,  79, 138,  97,  80, 201, 161,  39,  60,
        128,   2, 191, 103, 175,  81, 147,  67, 207, 116,  53, 134, 137, 122,
        177,  92, 141,  40,  62,  28, 148,  90, 117, 104, 178,  11,  48, 105,
        196, 163, 153,  12,  52,  36,  46, 158,  86,  99, 139, 188,  41, 114,
        197,  83, 157, 162, 144,   7,  72,  31,  87, 179,  64, 101, 146,  21,
        169, 150,  76, 120,  57, 142, 130,  37,  35, 125,  70,   8, 193,  18,
         95,  29,  58, 106,  55, 119,   6, 143, 111,  75,  47,  24,  45,   9,
        136, 185, 182, 189, 184, 167, 155,   1,  93,  91, 151,  27, 121, 170,
        165, 102,  51, 132, 107, 109, 124,  23, 154, 123,  50,   4, 200, 115,
         43,  59, 110,  88,  10,   3,  19,  32,  74, 108, 166,  68, 183,  71,
         82,  38, 133, 186,  77, 195,  73,  20,  16,  89, 129,  61,  85, 190,
        156, 174,  84, 118, 113,  98, 194, 202, 145,  17,  15, 1

In [41]:
masked_num = int(seq_len * 0.15)
random_num = int(0.1*masked_num)
keep_num = int(0.1*masked_num)
masked_num = masked_num - random_num - keep_num
# generate #masked_num indices from range 1 to batch_tokens.size(1)
shuffled_seqind = torch.randperm(seq_len)+1

In [47]:
keep_id = shuffled_seqind[0:masked_num].size()
random_id = shuffled_seqind[masked_num:masked_num+masked_num].size()
mask_id = shuffled_seqind[masked_num+masked_num:].size()

torch.Size([25])

In [55]:
torch.randint(1,26,(1,random_num)).size()

torch.Size([1, 3])

In [54]:
random_num

3